In [1]:
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error,mean_absolute_error
from sklearn.datasets import load_boston
boston = load_boston()

根据作业文件，lasso问题的迭代算法为$$(\beta^{t+1})_j=(1-\frac{\lambda\mu}{|\tilde{\beta_j^t}|})_{+}\tilde{\beta^t_j},j=1,\dots,p$$
其中$\tilde{\beta^t}=\beta^t-\mu\nabla f(\beta^t)$

我们设$$f(\beta)=\frac{1}{2}||y-x^T\beta||^2_2$$因此$$\nabla f(\beta^t)=x(x^T\beta-y)$$由此得到了迭代算法的显式公式。

接下来我们定义$(u)_+$函数、导数函数和示性函，然后写出迭代算法。

In [2]:
def vector_relu(u):
    for i in range(len(u)):
        u[i]=max(0,u[i,0])
    return u

def grad(x,beta,y):     
    return x.T.dot(x.dot(beta) - y)

def sign(x):
    n=x.shape[0]
    signal=np.zeros(n)
    for i in range(n):
        if x[i,0]!=0:
            signal[i]=x[i,0]/abs(x[i,0])
        else:
            signal[i]=0;
    return signal.reshape(n,1)

def lasso_proximal(X,y,lambda_lasso,max_iter,epsilon=1e-10):
    n = X.shape[1]
    beta = np.zeros((n,1))
    #mu设为0.0001
    mu = np.linspace(0.0001,0.0001,n).reshape(1,n)
    y = y.reshape(len(y),1)
    for i in range(max_iter):
        beta_j1 = beta - mu.T*grad(X,beta,y)
        beta_j1 = vector_relu(abs(beta_j1)-mu.T * lambda_lasso) * sign(beta_j1)
        #使用调整的相对收敛准则
        if (sum(abs(beta_j1 - beta))/(sum(abs(beta))+epsilon)) < epsilon:
            beta = beta_j1
            break
        else:
            beta = beta_j1
    return beta
    
    

接下来我们对数据集进行划分，为训练集和测试集，通过交叉验证，找到最优的$\lambda$，由于lasso是一种有偏方法，所以如果用MSE来评估效果是无效的，我选择的评价标准是BIC，因为BIC准则考虑了参数个数。

In [3]:
#对数据进行标准化
def normalization_X(x):
    return (x-x.mean(axis=0))/x.std(axis=0)

def normalization_y(y): 
    return (y-y.mean)/y.std  
#获取数据
X = boston.data
y = boston.target
X_train,X_test,y_train,y_test = train_test_split(X,y,test_size=0.3,random_state=2017110260)
X_train_n = normalization_X(X_train)
X_test_n = normalization_X(X_test)
y_train_n = normalization_X(y_train)
y_test_n = normalization_X(y_test)

In [5]:
#5折交叉验证选取lambda
num_folds = 5
lambda_choices = [pow(10, -3+0.1*i) for i in range(61)]
X_train_folds = []
y_train_folds = []
k_bics = {}
k_mses = {}
#划分数据集
X_train_folds = np.array_split(X_train_n, num_folds)
y_train_folds = np.array_split(y_train_n, num_folds)

for k in lambda_choices:
    bic = []
    mse = []
    for i in range(num_folds):
        Xtr = np.concatenate(X_train_folds[:i] + X_train_folds[i+1:])
        ytr = np.concatenate(y_train_folds[:i] + y_train_folds[i+1:])
        Xcv = X_train_folds[i]
        ycv = y_train_folds[i]
        #计算beta
        beta = lasso_proximal(Xtr,ytr,k,max_iter=10000)
        y_train_predict=Xcv.dot(beta)
        n=Xcv.shape[0]
        #计算BIC
        bic_predict = n*np.log(mean_squared_error(ycv,y_train_predict))+np.log(n)*np.sum(np.abs(sign(beta)))
        bic.append(bic_predict)
        #计算MSE
        mse_predict = mean_squared_error(ycv,y_train_predict)
        mse.append(mse_predict)
    k_bics[k] = bic
    k_mses[k] = mse


In [7]:
best_k=0
best_k_bic=0
best_k_mse=0
#找到最优lambda
for k in sorted(k_bics):
    avg_bic = sum(k_bics[k][:])/num_folds
    avg_mse = sum(k_mses[k][:])/num_folds
    if best_k_bic>avg_bic:
        best_k=k
        best_k_bic=avg_bic
        best_k_mse=avg_mse
print('最优超参数为'+str(best_k)+'，此时BIC为'+str(best_k_bic)+'，此时MSE为'+str(best_k_mse))

最优超参数为31.622776601683793，此时BIC为-62.2997794617605，此时MSE为0.36836264302460814


In [8]:
beta_final = lasso_proximal(Xtr,ytr,31.622776601683793 ,max_iter=10000)
beta_final

array([[-0.        ],
       [ 0.        ],
       [-0.        ],
       [ 0.02246276],
       [-0.        ],
       [ 0.26771667],
       [-0.        ],
       [-0.        ],
       [-0.        ],
       [-0.        ],
       [-0.14691206],
       [ 0.        ],
       [-0.43204338]])

我们可以看到，此时beta有9个系数都被成功压到0，这与lasso的思想相一致。

我们可以看到，对房价有显著影响的变量有是否有河道、每间住宅的平均房间数、城镇的学生与教师比例、人口状况下降，这四个变量显著相关。其中是否有河道、平均房间数的系数为正；城镇的学生与教师比例、人口状况下降的系数为负，这说明存在河道、平均房间数越多，教师与学生相对比例越大，人口越多的地区，房价越高。由于数据进行了标准化，因此系数绝对值的相对大小可以反应变量的重要性；人口数量、平均房间数、教师相对与学生比例的重要性递减。